## **[Append History]**  
- Add Language Filter for build_inference_set & inference Function

In [1]:
##### import wandb
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tokenizers, transformers
import os, sys, gc, time, random, warnings, math, json

from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.checkpoint import checkpoint

from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_cosine_schedule_with_warmup, DataCollatorWithPadding
from sklearn.model_selection import StratifiedGroupKFold
from tqdm.auto import tqdm
import cupy as cp
from cuml.metrics import pairwise_distances
from cuml.neighbors import NearestNeighbors
# from kaggle_secrets import UserSecretsClient
warnings.filterwarnings("ignore")
%env TOKENIZERS_PARALLELISM=false
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

env: TOKENIZERS_PARALLELISM=false


In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/stage3_multi-lingual-mpnet.pth
/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/__results__.html
/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/__notebook__.ipynb
/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/__output__.json
/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/custom.css
/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/wandb/run-20230313_113429-orp1r760/run-orp1r760.wandb
/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/wandb/run-20230313_113429-orp1r760/logs/debug.log
/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/wandb/run-20230313_113429-orp1r760/logs/debug-internal.log
/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/wandb/run-20230313_113429-orp1r760/files/wandb-summary.json
/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/wandb/run-20230313_113429-orp1r760/files/conda-environment.yaml
/kaggle/input/fold-0-lecr-re-rankerlm-m-lingual-train/wandb/run-20230313_113429

In [3]:
# # WandB Login => Copy API Key
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("wandb")

# !wandb login $secret_value_0

In [4]:
# # Epoch 10 Ver
# # paraphrase multi-lingual mpnet-base-v2
# class CFG1:
#     wandb, train, competition, seed = True, True, 'LECR', 42
#     device, gpu_id = torch.device('cuda' if torch.cuda.is_available() else 'cpu'), 0
#     # model
#     retriever = '/kaggle/input/paraphrase-multi-lingual-mpnet-stage3/paraphrase-multilingual-mpnet-base-v2-exp_fold0_epochs10'
#     # after Stage 3 running finish, upload our Real reranker model
#     reranker = '/kaggle/input/paraphrase-multi-lingual-mpnet-stage3/paraphrase-multilingual-mpnet-base-v2-exp_fold0_epochs10'
#     # tokenizer
#     retriever_tokenizer = AutoTokenizer.from_pretrained(retriever + '/tokenizer')
#     reranker_tokenizer = AutoTokenizer.from_pretrained(reranker + '/tokenizer')
#     # pooling
#     pooling = 'attention' # options: attention, mean, weightedlayer, concat
#     # Train/Validation
#     batch_size = 512    
#     top_n = 30 # for KNN
#     max_len = 256
#     n_fold = 5
#     gradient_checkpointing = True
#     num_workers = 1
#     threshold = 0.05 # for test dataset, maybe later change, # Best Threshold: 0.055

In [5]:
# # Epoch 20 Ver
# # paraphrase multi-lingual mpnet-base-v2
# class CFG1:
#     wandb, train, competition, seed = True, True, 'LECR', 42
#     device, gpu_id = torch.device('cuda' if torch.cuda.is_available() else 'cpu'), 0
#     # model
#     retriever = '/kaggle/input/stage1-epoch20-multi-lingual-mpnet/paraphrase-multilingual-mpnet-base-v2-exp_fold0_epochs10'
#     # after Stage 3 running finish, upload our Real reranker model
#     reranker = '/kaggle/input/stage1-epoch20-multi-lingual-mpnet/paraphrase-multilingual-mpnet-base-v2-exp_fold0_epochs10'
#     # tokenizer
#     retriever_tokenizer = AutoTokenizer.from_pretrained(retriever + '/tokenizer')
#     reranker_tokenizer = AutoTokenizer.from_pretrained(reranker + '/tokenizer')
#     # pooling
#     pooling = 'attention' # options: attention, mean, weightedlayer, concat
#     # Train/Validation
#     batch_size = 512    
#     top_n = 30 # for KNN
#     max_len = 256
#     n_fold = 5
#     gradient_checkpointing = True
#     num_workers = 1
#     threshold = 0.052 # for test dataset, maybe later change, # Best Threshold: 0.055

In [6]:
# Epoch 30 Ver
# paraphrase multi-lingual mpnet-base-v2
class CFG1:
    wandb, train, competition, seed = True, True, 'LECR', 42
    device, gpu_id = torch.device('cuda' if torch.cuda.is_available() else 'cpu'), 0
    # model
    retriever = '/kaggle/input/stage1-epoch30-multi-lingual-mpnet/paraphrase-multilingual-mpnet-base-v2-exp_fold0_epochs10'
    # after Stage 3 running finish, upload our Real reranker model
    reranker = '/kaggle/input/stage1-epoch30-multi-lingual-mpnet/paraphrase-multilingual-mpnet-base-v2-exp_fold0_epochs10'
    # tokenizer
    retriever_tokenizer = AutoTokenizer.from_pretrained(retriever + '/tokenizer')
    reranker_tokenizer = AutoTokenizer.from_pretrained(reranker + '/tokenizer')
    # pooling
    pooling = 'attention' # options: attention, mean, weightedlayer, concat
    # Train/Validation
    batch_size = 512    
    top_n = 30 # for KNN
    max_len = 256
    n_fold = 5
    gradient_checkpointing = True
    num_workers = 0
    threshold = 0.05

In [7]:
# Step 1.2 Class to Dictionary for WandB Train Experiment Tracking
def class2dict(f):
    return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

In [8]:
# Step 1.3 Seed Everything
def all_type_seed(CFG):
    os.environ['PYTHONHASHSEED'] = str(CFG.seed) # python Seed 
    random.seed(CFG.seed) # random module Seed
    np.random.seed(CFG.seed) # numpy module Seed
    torch.manual_seed(CFG.seed)

    torch.manual_seed(CFG.seed) # Pytorch CPU Random Seed Maker 
    torch.cuda.manual_seed(CFG.seed) # Pytorch GPU Random Seed Maker 
    torch.cuda.manual_seed_all(CFG.seed) # Pytorch Multi Core GPU Random Seed Maker 

    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False 
    torch.backends.cudnn.enabled = False
    
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() %2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
all_type_seed(CFG1)    
g = torch.Generator()
g.manual_seed(0)

In [9]:
# Breadcrumbs 
class Topic:
    def __init__(self, topic_id):
        self.id = topic_id

    @property
    def parent(self):
        parent_id = topics_df.loc[self.id].parent
        if pd.isna(parent_id):
            return None
        else:
            return Topic(parent_id)

    @property
    def ancestors(self):
        ancestors = []
        parent = self.parent
        while parent is not None:
            ancestors.append(parent)
            parent = parent.parent
        return ancestors

    @property
    def siblings(self):
        if not self.parent:
            return []
        else:
            return [topic for topic in self.parent.children if topic != self]

    @property
    def content(self):
        if self.id in correlations_df.index:
            return [ContentItem(content_id) for content_id in correlations_df.loc[self.id].content_ids.split()]
        else:
            return tuple([]) if self.has_content else []

    def get_breadcrumbs(self, separator=" ", include_self=False, include_root=True):
        ancestors = self.ancestors
        if include_self:
            ancestors = [self] + ancestors
        if not include_root:
            ancestors = ancestors[:-1]
        return separator.join(reversed([a.title for a in ancestors]))

    @property
    def children(self):
        return [Topic(child_id) for child_id in topics_df[topics_df.parent == self.id].index]

    def subtree_markdown(self, depth=0):
        markdown = "  " * depth + "- " + self.title + "\n"
        for child in self.children:
            markdown += child.subtree_markdown(depth=depth + 1)
        for content in self.content:
            markdown += ("  " * (depth + 1) + "- " + "[" + content.kind.title() + "] " + content.title) + "\n"
        return markdown

    def __eq__(self, other):
        if not isinstance(other, Topic):
            return False
        return self.id == other.id

    def __getattr__(self, name):
        return topics_df.loc[self.id][name]

    def __str__(self):
        return self.title
    
    def __repr__(self):
        return f"<Topic(id={self.id}, title=\"{self.title}\")>"


class ContentItem:
    def __init__(self, content_id):
        self.id = content_id

    @property
    def topics(self):
        return [Topic(topic_id) for topic_id in topics_df.loc[correlations_df[correlations_df.content_ids.str.contains(self.id)].index].index]

    def __getattr__(self, name):
        return content_df.loc[self.id][name]

    def __str__(self):
        return self.title
    
    def __repr__(self):
        return f"<ContentItem(id={self.id}, title=\"{self.title}\")>"

    def __eq__(self, other):
        if not isinstance(other, ContentItem):
            return False
        return self.id == other.id

    def get_all_breadcrumbs(self, separator=" >> ", include_root=True):
        breadcrumbs = []
        for topic in self.topics:
            new_breadcrumb = topic.get_breadcrumbs(separator=separator, include_root=include_root)
            if new_breadcrumb:
                new_breadcrumb = new_breadcrumb + separator + self.title
            else:
                new_breadcrumb = self.title
            breadcrumbs.append(new_breadcrumb)
        return breadcrumbs

In [10]:
# Step 2.1 Load Data & Preprocess Function
# add Feature 1: topic title same as content title
data_path = '/kaggle/input/learning-equality-curriculum-recommendations/'
topics_df = pd.read_csv(data_path + 'topics.csv')
content_df = pd.read_csv(data_path + 'content.csv')
correlations_df = pd.read_csv(data_path + 'correlations.csv') # for topic tree
sample_submission = pd.read_csv(data_path + 'sample_submission.csv') # for submission

# add Feature 1: topic title same as content title
# ascending
predicts = []
for topic_id in sample_submission['topic_id']:
    title = topics_df.query(f"id=='{topic_id}'")['title'].iloc[0]
    p = content_df[content_df['title'] == title]['id'].tolist()
    predicts.append(p)

topic_ids = topics_df.id.to_list()
topics_df.set_index(keys='id', inplace=True)
topics_df.insert(0, 'id', topic_ids)

# Remove category == source topics => Mention in Host's Tips
# will be no source data in future testing
# topics_df = topics_df[topics_df['category'] != 'source']

# title NaN
topics_df['title'].fillna("", inplace = True)
content_df['title'].fillna("", inplace = True)

# descriptions NaN
topics_df['description'].fillna("", inplace = True)
content_df['description'].fillna("", inplace = True)

# text NaN
content_df['text'].fillna("", inplace=True) 
# topic-description
for i in tqdm(range(len(topics_df))):
    context = Topic(topics_df.iloc[i, 0]).get_breadcrumbs()
    topic_feature = topics_df.iloc[i, 2] # select description feature
    if len(topic_feature) != 0:
        try:
            idx = topic_feature.index('.')
            first_sentence = topic_feature[0:idx] # first sentence in Full sentence
            if len(first_sentence.split(' ')) <= 40:
                topics_df.iloc[i,1] = topics_df.iloc[i,1][0:40] + ' <|=t_sep=|> ' + first_sentence + ' <|=t_sep=|> ' + context
            else:
                first_sentence = first_sentence.split(' ')[0:40]
                first_sentence = ' '.join(first_sentnce)
                topics_df.iloc[i,1] = topics_df.iloc[i,1][0:40] + ' <|=t_sep=|> ' + first_sentence + ' <|=t_sep=|> ' + context

        except:
            if len(topic_feature.split(' ')) <= 40:
                topics_df.iloc[i,1] = topics_df.iloc[i,1][0:40] + ' <|=t_sep=|> ' + topic_feature + ' <|=t_sep=|> ' + context
            else:
                topic_feature = topic_feature.split(' ')[0:40]
                topic_feature = ' '.join(topic_feature)
                topics_df.iloc[i,1] = topics_df.iloc[i,1][0:40] + ' <|=t_sep=|> ' + topic_feature + ' <|=t_sep=|> ' + context

    else: # case: NaN
        topics_df.iloc[i,1] = topics_df.iloc[i,1][0:40] + ' <|=t_sep=|> ' + context

# content_df-description or text
for i in tqdm(range(len(content_df))):
    if len(content_df.iloc[i,2]) != 0: 
        content_df_feature = content_df.iloc[i,2]
        try:
            idx = content_df_feature.index('.')
            first_sentence = content_df_feature[0:idx]
            if len(first_sentence.split(' ')) <= 40:
                content_df.iloc[i,1] = content_df.iloc[i,1] + ' <|=t_sep=|> ' + first_sentence
            else:
                first_sentence = first_sentence.split(' ')[0:40]
                first_sentence = ' '.join(first_sentence)
                content_df.iloc[i,1] = content_df.iloc[i,1] + ' <|=t_sep=|> ' + first_sentence
            continue
        except:
            if len(content_df_feature.split(' ')) <= 40:
                content_df.iloc[i,1] = content_df.iloc[i,1] + ' <|=t_sep=|> ' + content_df_feature
            else:
                content_df_feature = content_df_feature.split(' ')[0:40]
                content_df_feature = ' '.join(content_df_feature)
                content_df.iloc[i,1] = content_df.iloc[i,1] + ' <|=t_sep=|> ' + content_df_feature
            continue

    if len(content_df.iloc[i,2]) == 0 and len(content_df.iloc[i,4]) != 0:
        content_df_feature = content_df.iloc[i,4]
        try:
            idx = content_df_feature.index('.')
            first_sentence = content_df_feature[0:idx]
            if len(first_sentence.split(' ')) <= 40:
                content_df.iloc[i,1] = content_df.iloc[i,1] + ' <|=t_sep=|> ' + first_sentence
            else:
                first_sentence = first_sentence.split(' ')[0:40]
                first_sentence = ' '.join(first_sentence)
                content_df.iloc[i,1] = content_df.iloc[i,1] + ' <|=t_sep=|> ' + first_sentence
            continue
        except:
            if len(content_df_feature.split(' ')) <= 40:
                content_df.iloc[i,1] = content_df.iloc[i,1] + ' <|=t_sep=|> ' + content_df_feature
            else:
                content_df_feature = content_df_feature.split(' ')[0:40]
                content_df_feature = ' '.join(content_df_feature)
                content_df.iloc[i,1] = content_df.iloc[i,1] + ' <|=t_sep=|> ' + content_df_feature
            continue
    else:
        continue
        
# Sort by title length to make inference faster: collate_fn
topics_df['length'] = topics_df['title'].apply(lambda x: len(x))
content_df['length'] = content_df['title'].apply(lambda x: len(x))
topics_df.sort_values('length', inplace = True)
content_df.sort_values('length', inplace = True)

# Reset index
topics_df.reset_index(drop = True, inplace = True)
content_df.reset_index(drop = True, inplace = True)

topics_df = topics_df.merge(sample_submission, how = 'inner', left_on = 'id', right_on = 'topic_id')
topics_df.drop(['description', 'channel', 'category', 'level', 'parent', 'length', 'content_ids', 'topic_id'], axis = 1, inplace = True)
content_df.drop(['description', 'kind', 'text', 'copyright_holder', 'license', 'length'], axis = 1, inplace = True)

print(' ')
print('-' * 50)
print(f"topics.shape: {topics_df.shape}")
print(f"content.shape: {content_df.shape}")

  0%|          | 0/76972 [00:00<?, ?it/s]

  0%|          | 0/154047 [00:00<?, ?it/s]

 
--------------------------------------------------
topics.shape: (5, 4)
content.shape: (154047, 3)


In [11]:
topics_list = topics_df.title.to_list()
content_list = content_df.title.to_list()

# tokenizing topic
tokens_len, topic_token = [], [] 
for topic in tqdm(topics_list):
    inputs = CFG1.retriever_tokenizer.encode_plus(
        topic,
        return_tensors = None,
        add_special_tokens = False,
        truncation = True,
        max_length = 192,
        return_token_type_ids = True,
        return_attention_mask = True    
    )
    topic_token.append(CFG1.retriever_tokenizer.decode(inputs.input_ids))
    tokens_len.append(len(inputs.input_ids))

# tokenizing content
tokens_len, content_token = [], []
for content in tqdm(content_list):
    inputs = CFG1.retriever_tokenizer.encode_plus(
        content,
        return_tensors = None,
        add_special_tokens = False,
        truncation = True,
        max_length = 64,
        return_token_type_ids = True,
        return_attention_mask = True    
    )
    content_token.append(CFG1.retriever_tokenizer.decode(inputs.input_ids))
    tokens_len.append(len(inputs.input_ids))
    
topics_df.title, content_df.title = topic_token, content_token
topics_df.reset_index(drop=True, inplace=True), content_df.reset_index(drop=True, inplace=True)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/154047 [00:00<?, ?it/s]

(None, None)

In [12]:
# Step 2.2 Custom DataSet for Retriever Model
class LECRDataset(Dataset):
    def __init__(self, df, CFG):
        super().__init__()
        self.text = df['title'].values
        self.cfg = CFG
        
    def tokenizing(self, text_data):
        inputs = self.cfg.retriever_tokenizer.encode_plus(text_data, 
                                                          return_tensors=None, # if true, tf.tensor, pt.tensor, numpy
                                                          add_special_tokens=True,
                                                          truncation=True,
                                                          max_length=self.cfg.max_len)
        for k, v in inputs.items():
            inputs[k] = torch.tensor(v, dtype=torch.long)
        
        return inputs
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        inputs = self.tokenizing(self.text[idx]) # tokenizing
        return inputs

In [13]:
# Step 2.3 Pooling => Attention, Mean, WeightedLayer, Concat
# WeightedLayer & Concat => https://www.kaggle.com/code/yuiwai/lecr-stsb-roberta-base
# Attention pooling
class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(nn.Linear(in_dim, in_dim),
                                       nn.LayerNorm(in_dim),
                                       nn.GELU(),
                                       nn.Linear(in_dim, 1),)

    def forward(self, last_hidden_state, attention_mask):
        w = self.attention(last_hidden_state).float()
        w[attention_mask==0]=float('-inf')
        w = torch.softmax(w,1)
        attention_embeddings = torch.sum(w * last_hidden_state, dim=1)
        return attention_embeddings

# Mean Pooling
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

# WeightedLayer Pooling
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, features):
        ft_all_layers = features['all_layer_embeddings']
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]

        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        features.update({'token_embeddings': weighted_average})
        return features

# Concat Pooling
class ConcatPooling(nn.Module):
    def __init__(self, backbone_config, pooling_config):
        super(ConcatPooling, self, ).__init__()

        self.n_layers = pooling_config.n_layers
        self.output_dim = backbone_config.hidden_size*pooling_config.n_layers

    def forward(self, inputs, backbone_outputs):
        all_hidden_states = get_all_hidden_states(backbone_outputs)
        concatenate_pooling = torch.cat([all_hidden_states[-(i + 1)] for i in range(self.n_layers)], -1)
        concatenate_pooling = concatenate_pooling[:, 0]
        return concatenate_pooling

In [14]:
# Step 2.4 Custom Model for Retriever Stage(LM)
class RetrieverLM(nn.Module):
    def __init__(self, CFG):
        super().__init__()
        self.cfg = CFG
        self.auto_cfg = AutoConfig.from_pretrained(CFG.retriever + '/config')
        self.backbone = AutoModel.from_pretrained(CFG.retriever + '/model',
                                                  config = self.auto_cfg)
        self.pooling = AttentionPooling(self.auto_cfg.hidden_size)
        
    def forward(self, inputs):
        outputs = self.backbone(**inputs) # inputs from LECRDataset
        last_hidden_state = outputs.last_hidden_state
        embedding = self.pooling(last_hidden_state,
                                 inputs['attention_mask'])
        return embedding

In [15]:
# Step 2.5 Fuction for extract Embedding, Get Accuracy Score
def extract_embeddings(loader_dataset, model, device):
    model.eval() # NOT Train
    preds = []
    for step, inputs in enumerate(tqdm(loader_dataset)):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.to('cpu').numpy())
    preds = np.concatenate(preds)
    return preds

def accuracy_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    int_true = np.array([len(x[0] & x[1]) / len(x[0]) for x in zip(y_true, y_pred)])
    return round(np.mean(int_true), 5)

def f2_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    tp = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    fp = np.array([len(x[1] - x[0]) for x in zip(y_true, y_pred)])
    fn = np.array([len(x[0] - x[1]) for x in zip(y_true, y_pred)])
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f2 = tp / (tp + 0.2 * fp + 0.8 * fn)
    return round(f2.mean(), 4)

In [16]:
# Step 2.6 Build Schema by Retriever Stage(LM)
def build_training_set(topics, content, cfg):
    # Create lists for training
    topics_ids, content_ids = [], []
    title1, title2 = [], []
    topics_languages, content_languages, has_contents = [], [], []
    
    # Iterate over each topic
    for k in tqdm(range(len(topics))):
        row = topics.iloc[k]
        topics_id = row['id']
        topics_language = row['language']
        topics_title = row['title']
        predictions = row['predictions'].split(' ')
        has_content = row['has_content']
        for pred in predictions:
            content_title = content.loc[pred, 'title']
            content_language = content.loc[pred, 'language']
            topics_ids.append(topics_id)
            content_ids.append(pred)
            title1.append(topics_title)
            title2.append(content_title)
            topics_languages.append(topics_language)
            content_languages.append(content_language)
            has_contents.append(has_content)

    
    # Build training dataset
    test = pd.DataFrame(
        {'topics_ids': topics_ids, 
         'content_ids': content_ids, 
         'title1': title1, 
         'title2': title2,
         'topic_language': topics_languages, 
         'content_language': content_languages,
         'has_contents': has_contents
        }
    )
    # Release memory
    del topics_ids, content_ids, title1, title2, topics_languages, content_languages, has_contents
    gc.collect()
    
    return test

In [17]:
# Step 2.7 Clustering: KNN, Cosine Similarity
def get_neighbors(topics, content, cfg):
    # Create topics, content dataset
    topics_dataset, content_dataset = LECRDataset(topics, cfg), LECRDataset(content, cfg)
    
    # Create topics, content DataLoaders
    topics_loader = DataLoader(
        topics_dataset, 
        batch_size = cfg.batch_size, 
        shuffle = False, 
        collate_fn = DataCollatorWithPadding(tokenizer = cfg.retriever_tokenizer,
                                             padding = 'longest'),
        num_workers = cfg.num_workers, 
        pin_memory = True, 
    )
    content_loader = DataLoader(
        content_dataset, 
        batch_size = cfg.batch_size, 
        shuffle = False, 
        collate_fn = DataCollatorWithPadding(tokenizer = cfg.retriever_tokenizer,
                                             padding = 'longest'), # sort by length
        num_workers = cfg.num_workers, 
        pin_memory = True,
        )

    # Create unsupervised model to extract embeddings
    model = RetrieverLM(cfg)
    model.to(device)

    # Predict topics
    topics_preds = extract_embeddings(topics_loader, model, device)
    content_preds = extract_embeddings(content_loader, model, device)
    
    # Transfer predictions to gpu
    topics_preds_gpu = cp.array(topics_preds)
    content_preds_gpu = cp.array(content_preds)

    # Release memory
    torch.cuda.empty_cache()
    del topics_dataset, content_dataset, topics_loader, content_loader, topics_preds, content_preds
    gc.collect()
    
    # KNN model
    print(' ')
    print('Training KNN model...')
    neighbors_model = NearestNeighbors(n_neighbors = cfg.top_n, metric = 'cosine')
    neighbors_model.fit(content_preds_gpu)
    indices = neighbors_model.kneighbors(topics_preds_gpu, return_distance = False)
    predictions = []
    for k in tqdm(range(len(indices))):
        pred = indices[k]
        p = ' '.join([content.loc[ind, 'id'] for ind in pred.get()])
        predictions.append(p)
    topics['predictions'] = predictions
    
    # Release memory
    del topics_preds_gpu, content_preds_gpu, neighbors_model, predictions, indices, model
    gc.collect()
    return topics, content

In [18]:
# Make Testing Data
def preprocess_test(test):
    test['title1'].fillna("Title does not exist", inplace = True)
    test['title2'].fillna("Title does not exist", inplace = True)
    # Create feature column
    test['text'] = test['title1'] + '[SEP]' + test['title2']
    # Drop titles
    test.drop(['title1', 'title2'], axis = 1, inplace = True)
    # Sort so inference is faster
    test['length'] = test['text'].apply(lambda x: len(x))
    test.sort_values('length', inplace = True)
    test.drop(['length'], axis = 1, inplace = True)
    test.reset_index(drop = True, inplace = True)
    gc.collect()
    return test

In [19]:
# Loop for CFG1, CFG2....
# cfg_list = [CFG1, CFG2, CFG3]
# cfg_list = [CFG3, CFG4]
# cfg_list = [CFG1, CFG2]
cfg_list = [CFG1]
for idx, cfg in tqdm(enumerate(cfg_list)):
    # Init wandb
#     wandb.init(project="LECR", 
#                name=cfg.retriever,
#                config=class2dict(cfg),
#                group=cfg.retriever,
#                job_type="train",
#                entity = "qcqced")
#     wandb_config = wandb.config

    # Run nearest neighbors
    topics, content = get_neighbors(topics_df, content_df, cfg)
    content.set_index('id', inplace = True)

    # Build training set
    test = build_training_set(topics, content, cfg)
    temp_test = preprocess_test(test)
    print(f'Our training set has {len(temp_test)} rows')
    
#     wandb.finish()
    
    torch.cuda.empty_cache()
    del topics, content
    gc.collect()

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/301 [00:00<?, ?it/s]

 
Training KNN model...


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Our training set has 150 rows


## Create Candidate Finish

In [20]:
class AverageMeter(object):
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

In [21]:
# Step 2.2 Custom DataSet for Re-Ranker Model
class NewDataset(Dataset):
    def __init__(self, df, CFG):
        super().__init__()
        self.text = df['text'].values
        self.cfg = CFG
        
    def tokenizing(self, text_data):
        inputs = self.cfg.reranker_tokenizer.encode_plus(text_data, 
                                                         return_tensors=None, # if true, tf.tensor, pt.tensor, numpy
                                                         add_special_tokens=True,
                                                         truncation=True,
                                                         padding='max_length',
                                                         max_length=self.cfg.max_len)
        for k, v in inputs.items():
            inputs[k] = torch.tensor(v, dtype=torch.long)
        
        return inputs
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        inputs = self.tokenizing(self.text[idx]) # tokenizing
        return inputs

In [22]:
# Step 2.4 Custom Model for Re-Ranker Stage(LM)
class ReRankerLM(nn.Module):
    def __init__(self, CFG):
        super().__init__()
        self.cfg = CFG
        self.auto_cfg = AutoConfig.from_pretrained(CFG.reranker + '/config',
                                                   output_hidden_states = True)
        self.model = AutoModel.from_pretrained(CFG.reranker + '/model',
                                               config = self.auto_cfg)
        self.fc = nn.Linear(self.auto_cfg.hidden_size, 1)
        self._init_weights(self.fc)
        
        # pooling
        if self.cfg.pooling == 'attention':
            self.pooling = AttentionPooling(self.auto_cfg.hidden_size)
        elif self.cfg.pooling == 'mean':
            self.pooling = MeanPooling()
        elif self.cfg.pooling == 'weightedlayer':
            self.pooling = WeightedLayerPooling(self.auto_cfg.num_hidden_layers, layer_weights = None)

        # checkpointing
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
         
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.auto_cfg.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.auto_cfg.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def forward(self, inputs):
        outputs = self.model(**inputs) # inputs from LECRDataset
        last_hidden_state = outputs.last_hidden_state
        embedding = self.pooling(last_hidden_state,
                                 inputs['attention_mask'])
        output = self.fc(embedding)
        return output

In [23]:
# inference loop function
def inference_fn(loader_test, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(loader_test, total = len(loader_test))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().squeeze().to('cpu').numpy().reshape(-1))
    predictions = np.concatenate(preds)
    return predictions

In [24]:
# df: test dataframe, cfg: configuration
def inference(test, cfg):
    test_dataset = NewDataset(test, cfg)
    
    # DataLoader
    loader_test = DataLoader(
        test_dataset,
        batch_size = cfg.batch_size,
        shuffle = False,
        worker_init_fn=seed_worker,
        generator=g,
        num_workers = cfg.num_workers,
        pin_memory = True,
        drop_last = False,
    )
    # inference
    model = ReRankerLM(cfg)

    #model.load_state_dict(torch.load('/kaggle/input/lecr-re-rankerlm-multi-lingual-train/stage3_multi-lingual-mpnet.pth'))
    model.load_state_dict(torch.load('/kaggle/input/final-colab-2th-epoch-fold4/stage3_multi-lingual-mpnet.pth'))
    prediction = inference_fn(loader_test, model, cfg.device)

    # Release memory
    torch.cuda.empty_cache()
    del test_dataset, loader_test, model, #state
    gc.collect()
  
    # Use threshold
    test['prediction'] = prediction
    test['predictions'] = np.where(prediction > cfg.threshold, 1, 0)
    
    df = pd.DataFrame(columns=['topic_id', 'content_ids'])
    df['topic_id']= list(set(test.topics_ids.to_list()))
    
    test = test[test['has_contents'] == True]
    test1 = test[(test['predictions'] == 1) & (test['topic_language'] == test['content_language'])]
    test1 = test1.groupby(['topics_ids'])['content_ids'].unique().reset_index()
    test1['content_ids'] = test1['content_ids'].apply(lambda x: ' '.join(x))
    test1.columns = ['topic_id', 'content_ids']
    test_r = pd.merge(df, test1, how='outer', on='topic_id')
    test_r = test_r[['topic_id', 'content_ids_y']]
    test_r.rename(columns={'content_ids_y': 'content_ids'}, inplace=True)
    test_r.sort_values('topic_id', inplace=True)
    test_r.reset_index(drop=True, inplace=True)
    test_r = test_r.fillna('')
    
    # add same title feature
    test_predict = []
    for i in range(len(test_r)):
        test_predict.append([j for j in test_r.iloc[i,1].split()])
    for i in range(len(predicts)):
        for j in range(len(predicts[i])):
            if len(predicts[i]) == 0:
                continue
            if predicts[i][j] not in test_predict[i]:
                test_predict[i].append(predicts[i][j])
                continue
    test_r.content_ids = test_predict
    test_r['content_ids'] = test_r['content_ids'].apply(lambda x: ' '.join(x))
    test_r.sort_values('topic_id', inplace=True)
    test_r.reset_index(drop=True, inplace=True)
    test_r = test_r.fillna('')

    test_r.to_csv('submission.csv', index = False)
    return test_r

In [25]:
# Inference
test_r = inference(temp_test, CFG1)
test_r.head()

  0%|          | 0/1 [00:00<?, ?it/s]

,topic_id,content_ids
0,t_00004da3a1b2,
1,t_00068291e9a4,c_5e152a94376f c_85e7c0954384
2,t_00069b63a70a,
3,t_0006d41a73a8,
4,t_4054df11a74e,c_80dfc28d7bf2 c_81c0baea8ec5 c_8577c06c226a
